In [4]:
import pyspark
from datetime import datetime, timedelta


spark = pyspark.sql.SparkSession.builder.appName("auctions").getOrCreate()
df_auctions = spark.read.csv('auctions.csv', header=True)
df_installs = spark.read.csv('installs.csv', header=True)
rdd_auctions = df_auctions.rdd
rdd_installs = df_installs.rdd


In [8]:
rdd_auctions.take(5)

[Row(date=u'2019-04-23 18:58:00.842116', device_id=u'2564673204772915246', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:01.530771', device_id=u'4441121667607578179', ref_type_id=u'7', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:01.767562', device_id=u'7721769811471055264', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:02.363468', device_id=u'6416039086842158968', ref_type_id=u'1', source_id=u'0'),
 Row(date=u'2019-04-23 18:58:02.397559', device_id=u'1258642015983312729', ref_type_id=u'1', source_id=u'0')]

In [ ]:
rdd_auctions1 = rdd_auctions.map(lambda x: (x[1], datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f")))

In [ ]:
rdd_auctions2 = rdd_auctions1.map(lambda x: (x[0], x[1], x[1].strftime("%H"), x[1].strftime("%M")))

In [ ]:
def getRange(row):
    a = None
    for i in range(0,6):
        if i*10<=int(row[3]) and int(row[3])<=(i+1)*10:
            a = i
    return (row[0], row[1], row[2], row[3], a)

In [ ]:
rdd_auctions3 = rdd_auctions2.map(getRange)

In [ ]:
rdd_auctions3.take(1)

In [ ]:
rdd_auctions3

In [17]:
def toCSVLine(data):
    try:
        return ','.join(str(d)) for d in data)
    except:
        return ','.join('' for d in data)

In [ ]:
lines_train = rdd_auctions3.map(toCSVLine)
rdd_auctions3.repartition(1).saveAsTextFile('auction_with_range.csv')

In [ ]:
# Installs

In [5]:
df_installs = spark.read.csv('installs.csv', header=True)
rdd_installs = df_installs.rdd

In [6]:
rdd_installs1 = rdd_installs.map(lambda x: (datetime.strptime(x[0], "%Y-%m-%d %H:%M:%S.%f"), \
                                  x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13],\
                                           x[14], x[15], x[16], x[17]))

In [7]:
rdd_installs.take(1)

[Row(created=u'2019-04-24 06:23:29.495', application_id=u'1', ref_type=u'1494519392962156891', ref_hash=u'4716708407362582887', click_hash=None, attributed=u'False', implicit=u'True', device_countrycode=u'6287817205707153877', device_brand=None, device_model=u'3.739127126472163e+17', session_user_agent=u'adjust.com', user_agent=None, event_uuid=u'79837499-2f2a-4605-a663-e322f759424f', kind=u'app_open', wifi=None, trans_id=None, ip_address=u'4243443387795468703', device_language=u'3.3013777759776993e+18')]

In [8]:
rdd_installs2 = rdd_installs1.map(lambda x: (x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10], x[11], x[12], x[13],\
                                           x[14], x[15], x[16], x[17],x[0].strftime("%H"), x[0].strftime("%M")))

In [9]:
def getRange(row):
    a = None
    for i in range(0,6):
        if i*10<=int(row[19]) and int(row[19])<=(i+1)*10:
            a = i
    return (row[0], row[1], row[2], row[3], row[4], row[5],\
            row[6], row[7], row[8], row[9], row[10], row[11], row[12],\
            row[13], row[14], row[15], row[16], row[17], row[18], row[19], a)

In [10]:
rdd_installs3 = rdd_installs2.map(getRange)

In [11]:
rdd_installs3.take(1)

[(datetime.datetime(2019, 4, 24, 6, 23, 29, 495000),
  u'1',
  u'1494519392962156891',
  u'4716708407362582887',
  None,
  u'False',
  u'True',
  u'6287817205707153877',
  None,
  u'3.739127126472163e+17',
  u'adjust.com',
  None,
  u'79837499-2f2a-4605-a663-e322f759424f',
  u'app_open',
  None,
  None,
  u'4243443387795468703',
  u'3.3013777759776993e+18',
  '06',
  '23',
  2)]

In [12]:
def toCSVLine(data):
    try:
        return ','.join(str(d).replace(",", "") for d in data)
    except:
        return ','.join('' for d in data)

In [13]:
rdd_installs4 = rdd_installs3.map(toCSVLine)
rdd_installs4.repartition(1).saveAsTextFile('installs_with_range.csv')

In [14]:
import pandas as pd

In [18]:
installs_range = pd.read_csv('installs_with_range.csv')

/Users/emi/venv/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [21]:
auctions_range = pd.read_csv('auction_with_range.csv')

In [22]:
installs_range.head()

,2019-04-24 06:23:29.495000,1,1494519392962156891,4716708407362582887,None,False,True,6287817205707153877,None.1,3.739127126472163e+17,...,None.2,79837499-2f2a-4605-a663-e322f759424f,app_open,None.3,None.4,4243443387795468703,3.3013777759776993e+18,06,23,2
0,2019-04-24 02:06:01.032000,1.0,1.494519e+18,7.143569e+18,None,False,False,6.287817e+18,None,7.80553892759877e+18,...,None,None,None,None,None,4.724289e+18,3.3013777759776993e+18,2.0,6.0,0.0
1,2019-04-20 10:15:36.274000,1.0,1.494519e+18,5.230323e+18,None,False,True,6.287817e+18,None,8.355495513718673e+18,...,None,dda99e3c-9c4b-487d-891c-79f0a02cb4a8,app_open,None,None,8.291809e+18,4.060929664968129e+18,10.0,15.0,1.0
2,2019-04-20 21:56:47.151000,1.0,1.494519e+18,5.097164e+18,None,False,True,6.287817e+18,None,2.3557720913769155e+18,...,None,7010c3ce-0fcf-46c6-9be8-374cc0e20af4,app_open,None,None,4.006812e+18,3.3013777759776993e+18,21.0,56.0,5.0
3,2019-04-20 22:40:41.239000,1.0,1.494519e+18,6.328028e+18,None,False,False,6.287817e+18,None,6.156971151807135e+18,...,None,None,None,None,None,3.386455e+18,3.3013777759776993e+18,22.0,40.0,4.0
4,2019-04-20 18:19:27.485000,1.0,1.494519e+18,7.522786e+18,None,False,True,6.287817e+18,None,6.208879341562586e+18,...,None,35e7504f-cd9b-4a78-b89f-7335a8bd735a,app_open,None,None,7.297343e+18,3.3013777759776993e+18,18.0,19.0,1.0


In [23]:
auctions_range.head()

,(u'2564673204772915246',datetime.datetime(2019,4,23,18,58,0,842116),'18','58',5)
0,(u'4441121667607578179',datetime.datetime(2019,4,23,18,58,1,530771),'18','58',5)
1,(u'7721769811471055264',datetime.datetime(2019,4,23,18,58,1,767562),'18','58',5)
2,(u'6416039086842158968',datetime.datetime(2019,4,23,18,58,2,363468),'18','58',5)
3,(u'1258642015983312729',datetime.datetime(2019,4,23,18,58,2,397559),'18','58',5)
4,(u'6707090658317158573',datetime.datetime(2019,4,23,18,58,2,675804),'18','58',5)
